In [7]:
import aocd
import dataclasses
import numpy as np
import enum

real_data = aocd.get_data(day=11, year=2022)
test_data = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

In [139]:
from typing import Sequence, Union
import itertools

@dataclasses.dataclass
class Monkey:
    """A monkey instance.
    
    Args:
        index: the monkey index. 'Monkey 0' has an index of 0.
        items: the starting items.
        operation: a string command.
        divisible_by: an int.
        test: a `dict` that takes `bool` and returns the monkey index
            to throw to.
    """
    index: int
    items: Sequence[int]
    operation: str
    divisible_by: int
    test_dict: dict
        
    def __post_init__(self):
        self.n_inspections = 0
        
    def inspect_and_throw_to(self) -> Sequence[int]:
        """Inspects, tests, and throws the top item.
        
        Returns:
            `throw_to`, item_value
        """
        self.n_inspections += 1
        old = self.items.pop(0)
        new = eval(self.operation)
        new_item = int(np.floor(new / 3.0))
        throw_to = self.test_dict[new_item % self.divisible_by == 0]
        return throw_to, new_item
    
    
def str_to_monkey(string: str) -> Monkey:
    """Converts the input strings to a monkey object.
    
    Args:
        string: the monkey making string.
        
    Returns:
        A constructed `Monkey`.
    """
    lines = string.split("\n")
    index = int(lines[0][:-1].split(" ")[-1])
    starting_items = [
        int(x) for x in 
        lines[1].split("  Starting items: ")[-1].split(", ")
    ]
    operation = lines[2].split("new = ")[-1]
    divisible_by = int(lines[3].split("divisible by ")[-1])
    if_true = int(lines[4].split("throw to monkey ")[-1])
    if_false = int(lines[5].split("throw to monkey ")[-1])
    test_dict = {True: if_true, False: if_false}
    return Monkey(
        index = index,
        items = starting_items,
        operation = operation,
        divisible_by = divisible_by,
        test_dict = test_dict
    )

def find_monkey(monkeys: Sequence[Monkey], index: int) -> Monkey:
    """Finds the `Monkey` by its index.
    
    Args:
        monkeys: the list of monkeys.
        index: the index you are looking for.
    Returns:
        The monkey you are looking for.
    """
    for monkey in monkeys:
        if monkey.index == index:
            return monkey
    
@dataclasses.dataclass
class SolverA:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.monkey_str = self.raw_data.split("\n\n")
        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        monkeys = [str_to_monkey(string) for string in self.monkey_str]
        for _ in range(20):
            for monkey in monkeys:
                while (len(monkey.items) != 0):
                    throw_to, item = monkey.inspect_and_throw_to()
                    target_monkey = find_monkey(monkeys, throw_to)
                    target_monkey.items.append(item)
                    
        all_n_inspections = []
        for monkey in monkeys:
            all_n_inspections.append(monkey.n_inspections)
        return np.prod(sorted(all_n_inspections)[-2:])

In [140]:
SolverA(test_data).find_answer()

10605

In [78]:
answer = SolverA(real_data).find_answer()
aocd.submit(answer, part="a", day=11, year=2022)

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


<Response [200]>

In [171]:
from typing import Sequence, Union
import itertools


@dataclasses.dataclass
class Item:
    """Defines an item. Keeps track of the remanders for given `factors`.
    
    Also does modulo operations for each and every `factor`.
    
    Args:
        value: the input value.
        factors: factors to keep track of
    """
    value: int
    factors: Sequence[int]
        
    def __post_init__(self):
        self.remainders = []
        for factor in self.factors:
            self.remainders.append(self.value % factor)
        
    def add(self, num: int):
        """Defines an add operation.
        
        Args:
            num: the number to add to.
        """
        for idx, (remainder, factor) in enumerate(zip(self.remainders, self.factors)):
            self.remainders[idx] = (remainder + num) % factor
        
    def multiply(self, num: int):
        """Defines a multiplication operation.
        
        Args:
            num: the number to multiply by.
        """
        for idx, (remainder, factor) in enumerate(zip(self.remainders, self.factors)):
            self.remainders[idx] = int((remainder * (num % factor)) % factor)
        
    def squared(self):
        """Defines a multiplication operation.
        
        Args:
            num: the number to multiply by.
        """
        for idx, (remainder, factor) in enumerate(zip(self.remainders, self.factors)):
            self.remainders[idx] = int((remainder ** 2) % factor)
            
    def divisible_by(self, num):
        if num not in self.factors:
            raise ValueError(f"Remainder {num} not found.")
            
        index = self.factors.index(num)
        return self.remainders[index] == 0
        
    def apply(self, operation: str):
        """Applies the operation depending on the input string.
        
        Args:
            string: the operation in string form.
        """
        if "old * old" in operation:
            self.squared()
        elif "old * " in operation:
            num = operation.split("old * ")[-1]
            self.multiply(int(num))
        elif "old + " in operation:
            num = operation.split("old + ")[-1]
            self.add(int(num))
        else:
            raise ValueError(f"Operation {operation} not found")
        
        
@dataclasses.dataclass
class Monkey:
    """A monkey instance.
    
    Args:
        index: the monkey index. 'Monkey 0' has an index of 0.
        items: the starting items.
        operation: a string command.
        divisible_by: an int.
        test: a `dict` that takes `bool` and returns the monkey index
            to throw to.
    """
    index: int
    items: Sequence[Item]
    operation: str
    divisible_by: int
    test_dict: dict
        
    def __post_init__(self):
        self.n_inspections = 0
        
    def inspect_and_throw_to(self) -> Sequence[int]:
        """Inspects, tests, and throws the top item.
        
        Returns:
            `throw_to`, item_value
        """
        self.n_inspections += 1
        item = self.items.pop(0)
        item.apply(self.operation)
        throw_to = self.test_dict[item.divisible_by(self.divisible_by)]
        return throw_to, item
    
def find_all_possible_factors(monkey_strings: Sequence[str]) -> Sequence[int]:
    """Finds all possible factors:
    
    Args:
        monkey_strings: the input strings to construct monkeys
        
    Returns:
        all factors in a list.
    """
    factors = []
    for monkey_string in monkey_strings:
        lines = monkey_string.split("\n")
        divisible_by = int(lines[3].split("divisible by ")[-1])
        factors.append(divisible_by)
    return factors
        
def str_to_monkey(string: str, factors: Sequence[int]) -> Monkey:
    """Converts the input strings to a monkey object.
    
    Args:
        string: the monkey making string.
        factors: all possible factors.
        
    Returns:
        A constructed `Monkey`.
    """
    lines = string.split("\n")
    index = int(lines[0][:-1].split(" ")[-1])
    starting_items = [
        Item(int(x), factors) for x in 
        lines[1].split("  Starting items: ")[-1].split(", ")
    ]
    operation = lines[2].split("new = ")[-1]
    divisible_by = int(lines[3].split("divisible by ")[-1])
    if_true = int(lines[4].split("throw to monkey ")[-1])
    if_false = int(lines[5].split("throw to monkey ")[-1])
    test_dict = {True: if_true, False: if_false}
    return Monkey(
        index = index,
        items = starting_items,
        operation = operation,
        divisible_by = divisible_by,
        test_dict = test_dict
    )

def find_monkey(monkeys: Sequence[Monkey], index: int) -> Monkey:
    """Finds the `Monkey` by its index.
    
    Args:
        monkeys: the list of monkeys.
        index: the index you are looking for.
    Returns:
        The monkey you are looking for.
    """
    for monkey in monkeys:
        if monkey.index == index:
            return monkey
    
@dataclasses.dataclass
class SolverB:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.monkey_str = self.raw_data.split("\n\n")
        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        factors = find_all_possible_factors(self.monkey_str)
        monkeys = [str_to_monkey(string, factors) for string in self.monkey_str]
        for _ in range(10000):
            for monkey in monkeys:
                while (len(monkey.items) != 0):
                    throw_to, item = monkey.inspect_and_throw_to()
                    target_monkey = find_monkey(monkeys, throw_to)
                    target_monkey.items.append(item)
        for monkey in monkeys:
            print(f"{monkey.n_inspections}")
        print("")
        all_n_inspections = []
        for monkey in monkeys:
            all_n_inspections.append(monkey.n_inspections)
        return np.prod(sorted(all_n_inspections)[-2:])

In [169]:
SolverB(test_data).find_answer()

52166
47830
1938
52013



2713310158

In [170]:
answer = SolverB(real_data).find_answer()
aocd.submit(answer, part="b", day=11, year=2022)

27438
26933
134479
117859
134482
80107
107875
134273

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 11! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>